# ZDDのグラフアルゴリズム
__　授業でせっかく習ったので、ZDDのグラフアルゴリズムを実際に`python`で書いてみた。11/11現在圧縮がうまくいっていない様子__

In [434]:
# 必要なモジュールのインポート
import collections
import copy

## フロンティア

In [435]:
def update_frontier(frontier,edge,edges,counter):
    """
    関数の概要：以前までのフロンティアと獲得エッジからフロンティアを更新する。フロンティアは全ての場合で等しいので、一括計算。
    　　　　　　一般に、１つの頂点で、「入次数の数ー１」だけフロンティアは増える。
    @param frontier ：以前までのフロンティア
    @param edge     ：新規獲得エッジ
    @param edges    ：エッジ集合
    @param counter  ：エッジの入次数を記録している辞書
    @return frontier：更新後のフロンティア
    """
    # 入次数が２で、新しい辺の終点の頂点番号が小さかったら追加するだけ。
    if counter[edge[0]] == 2 and edge[1] == [edges[i][1] for i in range(len(edges)) if edges[i][0] == edge[0]][0]:
        frontier.append(edge[1])
    # 入次数が２で、新しい辺の終点の頂点番号が大きかったら始点を削除して終点を追加する。
    elif counter[edge[0]] == 2 and edge[1] == [edges[i][1] for i in range(len(edges)) if edges[i][0] == edge[0]][1]:
        frontier.remove(edge[0])
        frontier.append(edge[1])
    # 入次数が１の場合は、始点を削除して終点を追加する。
    elif counter[edge[0]] == 1:
        frontier.remove(edge[0])
        frontier.append(edge[1])
    else:
        print("予想外のedge: {}です!!".format(edge))
    
    # 重複を削除する。
    new_frontier = list(set(frontier))
    return new_frontier

In [436]:
def print_frontier(edges, num=False):
    """
    関数の概要：フロンティアの遷移を見る。
    　　　　　　番号を指定すればそのタイミングだけを表示する。
    @param edges：利用するエッジの情報
    @param num  ：指定する番号
    """
    frontier = [edges[0][0]]
    if not num: 
        num = [i for i in range(len(edges))] 
    else:
        num=[num-1]
    counter = collections.Counter([edge[0] for edge in edges])
    print("00: initial frontier:              {}".format(frontier))
    for i in range(len(edges)):          
        frontier = update_frontier(frontier, edges[i], edges, counter)
        if i in num:  
            print("{:0=2}: new_edge:{} frontier:{}".format(i+1,edges[i],frontier))

In [437]:
# 辺の集合
edges = [("e0","e1"),("e0","e2"),("e1","e3"),("e1","e4"),("e2","e5"),("e3","e5"),("e4","e5")]
# フロンティアの遷移を確認する。
print_frontier(edges)

00: initial frontier:              ['e0']
01: new_edge:('e0', 'e1') frontier:['e0', 'e1']
02: new_edge:('e0', 'e2') frontier:['e2', 'e1']
03: new_edge:('e1', 'e3') frontier:['e3', 'e1', 'e2']
04: new_edge:('e1', 'e4') frontier:['e3', 'e4', 'e2']
05: new_edge:('e2', 'e5') frontier:['e3', 'e4', 'e5']
06: new_edge:('e3', 'e5') frontier:['e4', 'e5']
07: new_edge:('e4', 'e5') frontier:['e5']


## mate配列

In [438]:
def update_mate_arr(mate, frontier, current_edges):
    """
    関数の概要：mate配列を更新する。
    @patam mate         ：以前までのmate配列（dict）
    @param frontier     ：更新後のフロンティア
    @param current_edges：獲得したエッジ集合
    @return new_mate    ：更新後のmate配列（dict）  
    """
    start = list(mate.keys())[0]
    end   = list(mate.keys())[-1]
    for f in range(len(frontier)):
        flag=False # ループに入るかの識別子
        loop = 0   # ループの回数を記録する。
        # 獲得したエッジ集合から、フロンティアの入次数を求める。
        in_edges  = [current_edges[i][0] for i in range(len(current_edges)) if current_edges[i][1] == frontier[f]]
        out_edges = [current_edges[i][1] for i in range(len(current_edges)) if current_edges[i][0] == frontier[f]]
        N_in  = len(in_edges)
        N_out = len(out_edges) # 出自数の数  
        # 入次数、出次数共に０だったら、頂点の名前になる。
        if N_in+N_out == 0:
            mate[frontier[f]] = frontier[f]
        # 入次数と出次数の和が２(=途中にある)の場合は０。    
        elif N_in + N_out>=2:
            mate[frontier[f]] = "0"
        # 入次数が１つだったら、逆端まで探索する。
        elif N_in == 1:
            edge = in_edges[0]
            b_edge = frontier[f] # １つ前のedgeを記録
            flag = True
        # 入次数は０だが、出次数が１の場合
        elif N_out == 1:
            edge = out_edges[0]
            b_edge = frontier[f]
            flag=True           
        # 探索を続ける。
        while flag and loop<len(current_edges): # 全てエッジを通っても終わらない＝ループが起きている。
            in_edges  = [current_edges[i][0] for i in range(len(current_edges)) if current_edges[i][1] == edge]
            out_edges = [current_edges[i][1] for i in range(len(current_edges)) if current_edges[i][0] == edge]
            N_in  = len(in_edges)  # 入次数の数
            N_out = len(out_edges) # 出自数の数                           
            # 端にたどり着いた場合
            if N_in + N_out == 1:
                mate[frontier[f]] = edge
                break
            # 端ではない場合は、端まで探索する。
            elif N_in == 2:
                if edge==end and loop!=0:
                    mate[frontier[f]] == "1"
                    break
                tmp = edge
                edge = [in_edges[i] for i in range(len(in_edges)) if in_edges[i]!=b_edge][0]
                b_edge = tmp
            elif N_out == 2:
                if edge== start and loop!=0:
                    mate[frontier[f]] == "1"
                    break
                tmp = edge
                edge = [out_edges[i] for i in range(len(out_edges)) if out_edges[i]!=b_edge][0]
                b_edge = tmp
            elif N_in==1 and N_out==1:
                tmp_edges = in_edges+out_edges
                tmp = edge
                edge = [tmp_edges[i] for i in range(len(tmp_edges)) if tmp_edges[i]!=b_edge][0]
                b_edge = tmp  
            loop+=1
        # 「エッジの数だけ遡っても端につかない」＝「ループができている」ため、削除する。
        if loop!=0 and loop==len(current_edges):
            for k in range(len(frontier)):
                mate[frontier[k]] = "0"
            return mate
    return mate

In [439]:
def print_matearr(vertex_set, edges, check_edges, num=False):
    """
    関数の概要：ある辺を取得した時のmate配列の遷移を表示する。
    @param vertex_set ：グラフの頂点集合
    @param edges      ：グラフの辺の集合
    @param check_edges：調べたい辺の集合
    @param frontier   ：フロンティア
    """
    current_edges = [] # 取得したエッジの初期化。
    mate_dict = dict() # mate配列の初期化
    for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = vertex_set[i]
    counter = collections.Counter([edge[0] for edge in edges]) # カウンターの作成。
    frontier = vertex_set[:1]
    
    if not num: 
        num = [i for i in range(len(edges))] 
    else:
        num=[num-1]
    
    for i in range(len(edges)):
        frontier = update_frontier(frontier, edges[i], edges, counter) # フロンティアの更新
        if edges[i] in check_edges: # 入っているものだけを取る。
            current_edges.append(edges[i])
        mate_dict = update_mate_arr(mate_dict, frontier, current_edges)  
        if i in num:  
            print("{:0=2}: new_edge     :{} {}".format(i+1,edges[i],"◯" if edges[i] in check_edges else "×"))
            print("    mate array   :{}".format({k:v for k, v in mate_dict.items() if k in frontier}))
            print("    current edges:{}\n".format(current_edges))  

In [440]:
# 確認
# 辺の集合
edges = [("e0","e1"),("e0","e2"),("e1","e3"),("e1","e4"),("e2","e5"),("e3","e5"),("e4","e5")]
# 頂点の集合
vertex_set = ["e0","e1","e2","e3","e4","e5"]
check_edges = [("e0","e2"),("e1","e3"),("e1","e4")]
print_matearr(vertex_set, edges, check_edges, 4)

04: new_edge     :('e1', 'e4') ◯
    mate array   :{'e2': 'e0', 'e3': 'e4', 'e4': 'e3'}
    current edges:[('e0', 'e2'), ('e1', 'e3'), ('e1', 'e4')]



<img src="../image/graphillion/sample2.png" width=50%>

## 一筆書き

__　始点から終点まで行くかを調べる__

In [441]:
def One_stroke(edges, start, end):
    """
    関数の概要：一筆書きできるかを判断する関数
    @param edges：調べたいエッジ
    @param start：始点
    @param end  ：終点
    @return：True or False
    """
    out_edges = [edges[i][1] for i in range(len(edges)) if edges[i][0] == start]
    edge = out_edges[0]
    b_edge = start
    count = 0; flag=False
    while count<len(edges):
        count += 1
        in_edges  = [edges[i][0] for i in range(len(edges)) if edges[i][1] == edge]
        out_edges = [edges[i][1] for i in range(len(edges)) if edges[i][0] == edge]
        N_in  = len(in_edges)  # 入次数の数
        N_out = len(out_edges) # 出自数の数
        if N_in + N_out == 1:
            flag = (edge == end) # 最後までたどり着いたかの確認。
            break
        elif N_in == 2:
            tmp = edge
            edge = [in_edges[i] for i in range(len(in_edges)) if in_edges[i]!=b_edge][0]
            b_edge = tmp
        elif N_out == 2:
            tmp = edge
            edge = [out_edges[i] for i in range(len(out_edges)) if out_edges[i]!=b_edge][0]
            b_edge = tmp
        elif N_in==1 and N_out==1:
            tmp_edges = in_edges+out_edges
            tmp = edge
            edge = [tmp_edges[i] for i in range(len(tmp_edges)) if tmp_edges[i]!=b_edge][0]
            b_edge = tmp
    return count==len(edges) and flag

In [442]:
# 確認
edges_set = [("e0","e1"),("e1","e3"),("e3","e4")]
print(One_stroke(edges_set, start="e0", end="e4"))
print(One_stroke(edges_set, start="e1", end="e4"))

True
False


__　パス探索の途中に、一本道だけで構成されているかを調べる__

In [443]:
def how_many_edges(edges, start):
    """
    関数の概要：与えられた場所から端まで行く時にどれだけエッジを通るかを返す関数
    @param edges：辺の集合
    @param start：
    """
    out_edges  = [edges[i][1] for i in range(len(edges)) if edges[i][0] == start]
    in_edges   = [edges[i][0] for i in range(len(edges)) if edges[i][1] == start]
    both_edges = out_edges+in_edges
    edge = both_edges[0]
    b_edge = start
    count=0
    # countの回数がedges数と同じでも到達しない＝ループがある、のため。
    while count<=len(edges):
        count += 1
        in_edges  = [edges[i][0] for i in range(len(edges)) if edges[i][1] == edge]
        out_edges = [edges[i][1] for i in range(len(edges)) if edges[i][0] == edge]
        N_in  = len(in_edges)  # 入次数の数
        N_out = len(out_edges) # 出自数の数
        if N_in + N_out == 1:  # 最後までたどり着いた。
            break
        elif N_in == 2:
            tmp = edge
            edge = [in_edges[i] for i in range(len(in_edges)) if in_edges[i]!=b_edge][0]
            b_edge = tmp
        elif N_out == 2:
            tmp = edge
            edge = [out_edges[i] for i in range(len(out_edges)) if out_edges[i]!=b_edge][0]
            b_edge = tmp
        elif N_in==1 and N_out==1:
            tmp_edges = in_edges+out_edges
            tmp = edge
            edge = [tmp_edges[i] for i in range(len(tmp_edges)) if tmp_edges[i]!=b_edge][0]
            b_edge = tmp
    return count

In [528]:
def One_stroke_with_frontier(edges, mate_arr, frontier, start):
    """
    関数の概要：一筆書きの集合かを判断する関数（ループを除く）
    @param edges     ：調べたいエッジ
    @param mate_arr  ：mate配列
    @param frontier  ：フロンティア集合
    @param start     ：始点
    @return：True or False
    """
    # フロンティアがkeyとなっているmate配列
    frontier_mate_dict = dict([(k,v) for k,v in mate_arr.items() if k in frontier])
    # フロンティア同士が繋がっているものの片方のフロンティアを残す。また、e0に繋がっているフロンティアも除く。
    together = [k for k,v in frontier_mate_dict.items() if (k in frontier) and (k in frontier_mate_dict.values()) and k<v and k!=start]
    out_edges = [edges[i][1] for i in range(len(edges)) if edges[i][0] == start]
    if len(out_edges)!=1: return False
    edge = out_edges[0]
    b_edge = start
    count = how_many_edges(edges, start)
    # フロンティア同士が繋がっているものがあれば、それも考える。
    for i in range(int(len(together))):
        count += how_many_edges(edges, together[i])
    
    return count==len(edges)

In [529]:
edges = [("e0","e3"),("e3","e6"),("e1","e4"),("e1","e2"),("e2","e5")]
mate_arr = dict([("e4","e5"),("e5","e4"),("e6","e0")])
frontier = ["e4","e5","e6"]
start = "e0"
One_stroke_with_frontier(edges, mate_arr, frontier, start)

True

In [530]:
edges = [("e0","e3"),("e3","e6"),("e1","e4"),("e1","e2"),("e2","e5"),("e4","e5")]
mate_arr = dict([("e4","e5"),("e5","e4"),("e6","e0")])
frontier = ["e4","e5","e6"]
start = "e0"
One_stroke_with_frontier(edges, mate_arr, frontier, start)

False

## 既に無理なものを取り除く

In [531]:
def Flatten_dual(dual_list):
    """
    関数の概要：二重リストを一重にする関数
    """
    return [element for inner_list in dual_list for element in inner_list]

In [532]:
def Remove_impossible(edges, frontier, start):
    """
    関数の概要：フロンティアに一続きで繋がっているかを確かめる。ただし、ループが含まれてしまう。
    @param edges   ：取得している辺の集合
    @param frontier：その時のフロンティア
    @param start   ：開始点(e0)
    """
    edge_elements = Flatten_dual(edges)
    counter = collections.Counter([edge for edge in edge_elements])
    check_list = [k for k, v in counter.items() if k not in frontier]
    if start in check_list: check_list.remove(start)
    for i in range(len(check_list)):
        if counter[check_list[i]] == 1:
            return False
    return True

In [533]:
# 確認
print(Remove_impossible([("e0","e3"),("e3","e4"),("e1","e2")], ["e4","e5","e6"], "e0"))
print(Remove_impossible([("e0","e3"),("e3","e4")], ["e4","e5","e6"], "e0"))

False
True


## メインプログラム

In [569]:
def rootnum(vertex_set, edges, flag=False):
    """
    関数の概要：ルートの総数を調べる。
    @param vertex_set ：グラフの頂点集合
    @param edges      ：グラフの辺の集合
    @param flag       ：エッジ集合を表示するか
    """
    start = vertex_set[0]
    end   = vertex_set[-1]
    frontier = vertex_set[:1] # 頂点集合の最初がスタートと仮定する。
    mate_dict = dict()        # mate配列の初期化。
    for i in range(len(vertex_set)): mate_dict[vertex_set[i]] = vertex_set[i]        
    edges_set = [[[],1,mate_dict]]# 初期化。
    counter = collections.Counter([edge[0] for edge in edges])
    answer = 0          # 求めたいルートの数
    answer_list = []
    
    edges_set = [[[],1,mate_dict]] # 初期化
    for i in range(len(edges)):
        # 新しいノードを獲得する処理
        frontier = update_frontier(frontier, edges[i], edges, counter) # フロンティアを更新する。
        edges_set_get = copy.deepcopy(edges_set) # 新規のエッジを獲得する方
        edges_set_not = copy.deepcopy(edges_set) # 新規のエッジを獲得しない方
        for j in range(len(edges_set_get)): 
            edges_set_get[j][0].append(edges[i])
        edges_set = edges_set_get+edges_set_not

        # mate配列を更新する
        for j in range(len(edges_set)):
            edges_set[j] = [edges_set[j][0],
                            edges_set[j][1],
                            update_mate_arr(edges_set[j][2],frontier,edges_set[j][0])]

        # [mate配列]をkey,[edgeの数,個数,edge]をvalueにする。
        result_dict = dict() # 圧縮で同じものを探すために辞書にしたが、しないのであればリストでも可能
        for j in range(len(edges_set)):
            mate_arr = edges_set[j][2] # 全てのmate配列(dictionaly型)
            mate_arr_frontier = dict([(k,v) for k,v in mate_arr.items() if k in frontier]) # フロンティア部分のみ
            mate_key = "".join([edges_set[j][2][frontier[f]] for f in range(len(frontier))])
            mate_id = (i,j)
            N_edges = len(edges_set[j][0])       # 獲得したエッジの数
            get_edges = edges_set[j][0]          # 獲得したエッジ
            num = edges_set[j][1]                # これより前で圧縮された等価なものの数
            # 始点がmate配列にないものはゴールできないので、ここで削除する。
            # また、すでに無理なものを取り除く。
            """
            ここで、既に不正解だとわかっているものを取り除く。
            ・フロンティア配列のmate配列のvalueにe0が存在しない。（これだとe0からエッジがないものも含まれる。）
            ・１つのノードから３本以上のエッジが出ている。
            ・ループができている。
            """
            if start in mate_arr_frontier.values():   # フロンティアのmate配列に始点が含まれないものを取り除く。
                if Remove_impossible(get_edges, frontier, start): # 既に無理なものを取り除く。
                    if start in Flatten_dual(get_edges): # 始点が獲得エッジに含まれているもののみを考える。（エラー処理）
                        if One_stroke_with_frontier(get_edges, mate_arr, frontier, start): # ループが複数ノードを取り除く。
                            if end in frontier and mate_arr[end] == start: # 終点がmate配列に入っており、始点と終点が繋がっていれば
                                if One_stroke(get_edges,start=start, end=end):                                
                                    if flag: answer_list.append(get_edges)       # 必要であれば最後まで行ったパスを記憶する。
                                    answer+=num
                                    continue
                        else:
                            continue
                    """
                    ここで、フロンティア部分のmate配列が等しいものを圧縮する。
                    """
                    # mate配列の同じものがあれば、エッジ数の少ないものを残して圧縮する。
                    if mate_key in result_dict:
                        result_dict[mate_key][2] += num # 同じものはまとめる。
                        # エッジ数が少なくなるのであれば、更新する。
                        if result_dict[mate_key][0] > N_edges:
                            result_dict[mate_key][0] = N_edges
                            result_dict[mate_key][1] = get_edges
                            result_dict[mate_key][3] = mate_arr

                    else:
                        result_dict[mate_key] = [N_edges, get_edges, num, mate_arr]
                    """
                    result_dict[mate_id] = [N_edges, get_edges, num, mate_arr]
                    """
                
        
        # 圧縮&NGを削除後を回収する。
        edges_set = []
        for k,v in result_dict.items():
            edges_set.append([v[1],v[2],v[3]])

    print("{} から {} までのルートの総数は {} 通りです。".format(start,end,answer))
    if flag: return answer_list

***
***

# 【実演】

In [570]:
# 辺の集合
edges = [("e0","e1"),("e0","e2"),("e1","e3"),("e1","e4"),("e2","e5"),("e3","e5"),("e4","e5")]
# 頂点の集合
vertex_set = ["e0","e1","e2","e3","e4","e5"]

<img src="../image/graphillion/sample.png" width=60%>

In [571]:
rootnum(vertex_set, edges, True)

e0 から e5 までのルートの総数は 3 通りです。


[[('e0', 'e2'), ('e2', 'e5')],
 [('e0', 'e1'), ('e1', 'e3'), ('e3', 'e5')],
 [('e0', 'e1'), ('e1', 'e4'), ('e4', 'e5')]]

***

In [572]:
# 辺の集合
edges = [("e0","e1"),("e0","e3"),("e1","e2"),("e1","e4"),("e3","e4"),("e3","e6"),("e2","e5"),("e4","e5"),("e4","e7"),("e6","e7"),("e5","e8"),("e7","e8")]
# 頂点の集合
vertex_set = ["e0","e1","e2","e3","e4","e5","e6","e7","e8"]

<img src="../image/graphillion/sample3.png" width=50%>

In [574]:
rootnum(vertex_set, edges, True)

e0 から e8 までのルートの総数は 12 通りです。


[[('e0', 'e3'),
  ('e3', 'e6'),
  ('e4', 'e5'),
  ('e4', 'e7'),
  ('e6', 'e7'),
  ('e5', 'e8')],
 [('e0', 'e3'), ('e3', 'e4'), ('e4', 'e5'), ('e5', 'e8')],
 [('e0', 'e3'), ('e3', 'e6'), ('e6', 'e7'), ('e7', 'e8')]]

***

In [575]:
# 辺の集合
edges = [("e0","e1"),("e0","e4"),("e1","e2"),("e1","e5"),("e4","e5"),("e4","e8"),("e2","e3"),("e2","e6"),("e5","e6"),("e5","e9"),("e8","e9"),("e8","e12"),("e3","e7"),("e6","e7"),("e6","e10"),("e9","e10"),("e9","e13"),("e12","e13"),("e7","e11"),("e10","e11"),("e10","e14"),("e13","e14"),("e11","e15"),("e14","e15")]
# 頂点の集合
vertex_set = ["e0","e1","e2","e3","e4","e5","e6","e7","e8","e9","e10","e11","e12","e13","e14","e15"]

<img src="../image/graphillion/sample4.png" width=50%>

In [576]:
rootnum(vertex_set, edges)

e0 から e15 までのルートの総数は 184 通りです。


<img src="../image/graphillion/oneesan.png" width=50%>

__　お姉さんがこれぐらい疲れてしまう $4×4$ ぐらいなら計算できたが、このままだと結構時間がかかってしまう。圧縮をするアルゴリズムを作成しなくてはならない。__

***

## 圧縮について

__　以下の２つはmate配列が一致してしまい、これを一致させることができない。どのようにして圧縮をすれば良いのだろうか…__

<img src="../image/graphillion/mate_compression.png" width=50%>

__　フロンティアと初期位置以外に出次数と入次数の和が２じゃないものがあったらそれあかん気がする。__

<img src="../image/graphillion/oneesan2.png" width=70%>